In [1]:
import pandas as pd
import math
import re


In [2]:
df=pd.read_csv("../data/CometLanding.csv", 
                dtype={"id_str": str, "in_reply_to_user_id_str": str, "from_user_id_str": str,
                        "in_reply_to_status_id_str": str, "user_followers_count": "Int64",
                        "user_friends_count": "Int64", "geo_coordinates" : str}, parse_dates=['created_at']
              )

In [3]:
df

,id_str,from_user,text,created_at,time,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,status_url,entities_str
0,540930409279008000,amika0078788556,RT @VersaTechnology: Congratulations @Philae20...,2014-12-05 18:07:14+00:00,05/12/2014 18:07,NaN,en,NaN,NaN,1297570116,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/3555068293...,425,113,http://twitter.com/amika0078788556/statuses/54...,"{""hashtags"":[{""text"":""Philae"",""indices"":[49,56..."
1,540930042700656000,ChrisDMarshall,CometWatch 2 December Â» Rosetta - ESA's comet...,2014-12-05 18:05:46+00:00,05/12/2014 18:05,NaN,en,NaN,NaN,42201214,NaN,"<a href=""https://dev.twitter.com/docs/tfw"" rel...",http://pbs.twimg.com/profile_images/228087980/...,347,1286,http://twitter.com/ChrisDMarshall/statuses/540...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
2,540929998388211000,MHuuskoL,RT @EUCouncil: After the #CometLanding - Astro...,2014-12-05 18:05:36+00:00,05/12/2014 18:05,NaN,en,NaN,NaN,574434762,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",http://pbs.twimg.com/profile_images/4973510588...,215,310,http://twitter.com/MHuuskoL/statuses/540929998...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
3,540929346878599000,SaraGomezAranci,RT @EUCouncil: After the #CometLanding - Astro...,2014-12-05 18:03:00+00:00,05/12/2014 18:03,NaN,fr,NaN,NaN,1057356487,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/5391705804...,741,310,http://twitter.com/SaraGomezAranci/statuses/54...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
4,540929229052203000,CBCDay6,RT @shaunmajumder: Feels good to be the @CBCDa...,2014-12-05 18:02:32+00:00,05/12/2014 18:02,NaN,en,NaN,NaN,177276897,NaN,"<a href=""http://twitter.com/download/android"" ...",http://pbs.twimg.com/profile_images/4590694303...,4331,2098,http://twitter.com/CBCDay6/statuses/5409292290...,"{""hashtags"":[{""text"":""MiniMansbridge"",""indices..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77314,532460149084930000,ABForScience,This means that the actual landing will be ar...,2014-11-12 09:09:26+00:00,12/11/2014 09:09,NaN,en,NaN,NaN,2333122519,NaN,"<a href=""http://www.tweetcaster.com"" rel=""nofo...",http://pbs.twimg.com/profile_images/4320695593...,155,212,http://twitter.com/ABForScience/statuses/53246...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
77315,532460148451581000,atieyK,RT @ObservingSpace: Weâ€™ve been waiting 10 ye...,2014-11-12 09:09:26+00:00,12/11/2014 09:09,NaN,en,NaN,NaN,58303715,NaN,"<a href=""https://twitter.com/download/android""...",http://pbs.twimg.com/profile_images/5057311637...,354,1088,http://twitter.com/atieyK/statuses/53246014845...,"{""hashtags"":[{""text"":""cometlanding"",""indices"":..."
77316,532460148254470000,j0nny5,RT @maxplanckpress: Accomazzo (flight director...,2014-11-12 09:09:26+00:00,12/11/2014 09:09,NaN,en,NaN,NaN,17914863,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/89991673/j...,92,63,http://twitter.com/j0nny5/statuses/53246014825...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
77317,532460148238090000,nsentse,7 hours of waiting #CometLanding,2014-11-12 09:09:26+00:00,12/11/2014 09:09,NaN,en,NaN,NaN,20434005,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/3693655850...,200,267,http://twitter.com/nsentse/statuses/5324601482...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."


In [4]:
len(df)

77319

In [5]:
df.dtypes

id_str                                    object
from_user                                 object
text                                      object
created_at                   datetime64[ns, UTC]
time                                      object
geo_coordinates                           object
user_lang                                 object
in_reply_to_user_id_str                   object
in_reply_to_screen_name                   object
from_user_id_str                          object
in_reply_to_status_id_str                 object
source                                    object
profile_image_url                         object
user_followers_count                       Int64
user_friends_count                         Int64
status_url                                object
entities_str                              object
dtype: object

In [6]:
df[df.duplicated()].head(20)

,id_str,from_user,text,created_at,time,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,status_url,entities_str
63,540787395940347000,ptmayes,RT @Johnwaynehawkes: If anything were to #Brea...,2014-12-05 08:38:57+00:00,05/12/2014 08:38,NaN,en,NaN,NaN,16086437,NaN,"<a href=""http://blackberry.com/twitter"" rel=""n...",http://pbs.twimg.com/profile_images/59254769/m...,15810,10790,http://twitter.com/ptmayes/statuses/5407873959...,"{""hashtags"":[{""text"":""BreakTheInternet"",""indic..."
71,540761145834209000,GamerGateAlpha,This did not disclose their ties to service pr...,2014-12-05 06:54:38+00:00,05/12/2014 06:54,NaN,en,NaN,NaN,2883539141,NaN,"<a href=""https://github.com/RemyPorter/AutoGat...",http://pbs.twimg.com/profile_images/5358767672...,5,1,http://twitter.com/GamerGateAlpha/statuses/540...,"{""hashtags"":[{""text"":""KathrinAltwegg"",""indices..."
83,540706624348774000,fullgracentruth,Watch Rosetta #CometLanding webcast on @livest...,2014-12-05 03:17:59+00:00,05/12/2014 03:17,NaN,en,NaN,NaN,990158232,NaN,"<a href=""https://dev.twitter.com/docs/tfw"" rel...",http://pbs.twimg.com/profile_images/3788000002...,43,219,http://twitter.com/fullgracentruth/statuses/54...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
89,540675346278805000,PurpleAly13,RT @adyrcz: Rosetta's Philae is on the comet! ...,2014-12-05 01:13:42+00:00,05/12/2014 01:13,NaN,en,NaN,NaN,1226993569,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/3318609506...,134,381,http://twitter.com/PurpleAly13/statuses/540675...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
95,540635598890237000,ansentorsan,RT @ESA_Rosetta: Moving back towards a 20 km o...,2014-12-04 22:35:45+00:00,04/12/2014 22:35,NaN,es,NaN,NaN,417218058,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",http://pbs.twimg.com/profile_images/1648843152...,8,29,http://twitter.com/ansentorsan/statuses/540635...,"{""hashtags"":[{""text"":""67P"",""indices"":[58,62]},..."
101,540599696817807000,glxp,From the #CometLanding to the Moon---join #GLX...,2014-12-04 20:13:06+00:00,04/12/2014 20:13,NaN,en,NaN,NaN,14208982,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/3788000001...,33953,4597,http://twitter.com/glxp/statuses/5405996968178...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
115,540556460443594000,acolliver615,"@ESA_Rosetta keep sending pictures, they're ep...",2014-12-04 17:21:17+00:00,04/12/2014 17:21,"loc: 39.2076777,-83.6107315",en,253536357,ESA_Rosetta,2260971996,540554322535534000,"<a href=""http://twitter.com/download/android"" ...",http://pbs.twimg.com/profile_images/5329781788...,56,364,http://twitter.com/acolliver615/statuses/54055...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
122,540523286715564000,GangjeeR,RT @ESA_Rosetta: Moving back towards a 20 km o...,2014-12-04 15:09:28+00:00,04/12/2014 15:09,NaN,en,NaN,NaN,1098858338,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/5072563303...,234,91,http://twitter.com/GangjeeR/statuses/540523286...,"{""hashtags"":[{""text"":""67P"",""indices"":[58,62]},..."
130,540517029619724000,Bekki_R_92,RT @ESA_Rosetta: Right now I think @philae2014...,2014-12-04 14:44:36+00:00,04/12/2014 14:44,NaN,en,NaN,NaN,159648324,NaN,"<a href=""http://twitter.com/download/android"" ...",http://pbs.twimg.com/profile_images/5208934443...,486,1821,http://twitter.com/Bekki_R_92/statuses/5405170...,"{""hashtags"":[{""text"":""67P"",""indices"":[118,122]..."
71410,532465601407909000,GMacademic,RT @newscientist: Hugs and cheers at ESA w new...,2014-11-12 09:31:06+00:00,12/11/2014 09:31,NaN,en,NaN,NaN,2350363310,NaN,"<a href=""http://twitter.com/download/android"" ...",http://pbs.twimg.com/profile_images/4589496259...,163,351,http://twitter.com/GMacade

In [7]:
df = df.drop_duplicates()
df = df.dropna(axis = 0, how = 'all')

In [8]:
reply_df = df[["in_reply_to_user_id_str","in_reply_to_screen_name","in_reply_to_status_id_str"]]

In [9]:
reply_df

,in_reply_to_user_id_str,in_reply_to_screen_name,in_reply_to_status_id_str
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
77314,NaN,NaN,NaN
77315,NaN,NaN,NaN
77316,NaN,NaN,NaN
77317,NaN,NaN,NaN


In [10]:
#reply
def isreply(tweet):
    if not pd.isna(tweet):
        reply =[]
        reply.append(tweet)
        print(tweet)
        return reply
        
#     if tweet["in_reply_to_user_id_str"] == 0:
#         return 0
#     return tweet["in_reply_to_user_id_str"]

#retweet
def is_rt(tweet):
    match = re.search("(^[R][T] )(@[A-Z,a-z]*)", tweet)
    if match != None:
#         print(tweet)
#         print("---------")
#         print("rt")
        return match.group(1)
    else:
#         print("tw")
        print("---------")
        print(tweet)
            

In [11]:
# replies
replies = reply_df["in_reply_to_user_id_str"].map(isreply)

11348282
19442947
253536357
253536357
64643056
33602654
1691515789
19402238
529722897
2294593507
41333964
253536357
66162716
59220577
253536357
1273487113
253536357
17842366
26030180
24129206
14526045
21436960
1969346438
253536357
253536357
2232463302
1438925534
15808765
472670056
2590415228
254659730
17384099
177594666
46959037
32520240
443080433
412396499
208442526
86070159
21436960
253536357
253536357
253536357
253536357
208442526
35233873
376790816
404818828
14800111
622113
1266281
2294593507
372434360
329600057
972651
208442526
168090600
246693449
27650674
208442526
1941637627
43024906
86336234
208442526
19903360
2868449663
47306483
371152085
2894572724
106442779
208442526
364423298
253536357
43024906
85751142
208442526
4620451
3108351
41712825
75052666
5402612
2254751592
629637909
21584325
110462645
208442526
5402612
14606395
208442526
75052666
1041442471
119473816
1435461
208442526
229632990
75052666
34800396
32498911
18505048
208442526
208442526
129673790
75052666
114085696
673

In [12]:
text = df["text"]

In [13]:
a = text.map(is_rt)

---------
CometWatch 2 December Â» Rosetta - ESA's comet chaser ##CometLanding ##CometWatch #lander #navcam  [...] http://t.co/cxgu1KKd3s
---------
USA Â» NEWS Â»  HOT NEWS 828 comet landing Philae touches down on the surface of a comet #cometlanding heyyouapp T... http://t.co/C4XSjBq7G2
---------
Feels good to be the @CBCDay6 champion beating #MiniMansbridge @davidcommon and @SusanKent #CometLanding #LetItSnow #MtRushmore
---------
After the #CometLanding - Astronaut @astro_luca discussed space policy with ministers today: http://t.co/ZjKAgpXhkt http://t.co/6R7yKrQh9I
---------
USA Â» NEWS Â»  HOT NEWS 534 comet landing The Rosetta comet landing has made history #cometlanding heyyouapp Cali... http://t.co/wfEi29flWz
---------
With @NASA_Orion today &amp; @Philae2014 @ESA_Rosetta 3 wks ago, 2014 is ending on a high note for #STEM lovers. #cometlanding #Orion #space
---------
Space has become exciting again #Orion #CometLanding #LunarMissionOne #NewHorizons
---------
November #CometLand

#WeCanLandOnACometButWeCant land on any of the other reindeer. #CometLanding
---------
* * * BREAKING NEWS * * * KATE UPTON SEX TAPE LEAKED * * * 23 Minutes Long - Go here to view: http://t.co/QaEqfrWBmL ~~~~ #CometLanding
---------
The #CometLanding has certainly marked a new chapter in the exploration of our Solar System - Mind blowing stuff from @esa
---------
http://t.co/XChks1SaSg #ESA_Rosetta @ESA_Rosetta @esaoperations #Vangelis #CometLanding A beautiful piece composed for this historic moment.
---------
Here is my soundtrack for @AmbitionTheFilm which was released ahead of @ESA_Rosetta #CometLanding https://t.co/XKTs2vXNDg
---------
today was important! mining comets will be important when our planet's resources become used up and depleted like my heart #CometLanding
---------
http://t.co/LlEMTMxULi. #CometLanding #rosettamission  Fascinating research !!!  Look what can b done !!!
---------
#CometLanding full tilt booter blue yellow, Get up to $600 free with your first deposit 

---------
There's a lot of men suddenly very interested in Dr Matt Taylor ... #CometLanding
---------
Wow so impressed by #cometlanding! This is taking star fighter to the next level!
---------
So in awe of everyone involved in the #CometLanding. You're all brilliant.
---------
æœèµ·ããŸã‚‰ã€ã‚¹ãƒžãƒ›ã®å£ç´™ãŒãƒ•ã‚£ãƒ¬ã ã£ãŸã€‚ç™ºéŸ³ã¯ fee-LAY ã£ã¦æ›¸ã„ã¦ã‚ã‚‹ã€‚ä»Šã¨ãªã£ã¦ã¯ãƒ•ã‚£ãƒ©ã‚¨ã§ã‚‚ãƒ•ã‚£ãƒ¬ã‚ªã§ã‚‚ãƒ”ãƒ­ã‚³ã§ã‚‚ä½•ã§ã‚‚ã„ã„ã‚„ã€‚ #CometLanding
---------
Massive controversy over trust vote in Maharashtra Assembly, 5 Congress MLAs suspended http://t.co/LSSSvTpvzC #MumbaiIn3Words
#CometLanding
---------
Who says cutting-edge rocket science can't be charming? #CometLanding bots flirt &amp; joke with each other on Twitter http://t.co/Aci7rea8yG
---------
Amazing images from the comet landing this morning. Go team Human! #philae #CometLanding
---------
Was watching the #CometLanding in physics today
---------
Richard Vaughan  @RichieVaughan88     :@CNN @

#CometLanding 

Cool
---------
I was initially impressed by landing on a comet but I'm over it now, that's what happens when you've seen Altidore score #cometlanding #safc
---------
Watch Rosetta #CometLanding webcast on @livestream: http://t.co/OGT1d5MQJE
---------
Seems i missed a #CometLanding. I imagine this kind of thing may one day make Bruce Willis obsolete.
---------
Â¡Aguanta @Philae2014! Â¡Aguanta! #rosettamission #rosettalanding #Rosetta #CometLanding
---------
Today's #fortunecookie says:  "Be persistent in pursuing the goals of your life."  #cometlanding #humanism
---------
Woke up to a strange sound today. Turns out it was just @Philae2014 landing outside. #CometLanding
---------
Love: lander's joyous robotic face looking back, to us, across the lonely distant blackness of space http://t.co/oD4TY0S075 #CometLanding
---------
Mi pare conveniente considerando che andare da Milano a Piacenza su un regionale scassato costa 6,50 â‚¬ #CometLanding http://t.co/syh8BZmaoN
-------

@OndaCero_es #CometLanding si fueron los cometas los que trajeron la vida a la tierra (aminoÃ¡cidos y H2O), si pero... vida Â¿desde donde? y 2
---------
TT CHILE 16:45
1.#TeSigoyTeSaludo
2.#CopaAmerica2015
3.#OReilly
4.#CometLanding
5.#LasTardesYaEstanPara
6.Rosetta
7.Copa AmÃ©rica
8.Philae
---------
What if you had to come out SMART the way you come out gay? Watch: http://t.co/H7OBZ3Te5L #cometlanding #wecanlandonacometbutwecant 10!q2
---------
TT COLOMBIA 15:45
1.#AdmitoQue
2.#UstedDeberia
3.#Forever20
4.#QuieroUnXperiaZ3
5.#CometLanding
6.Rosetta
7.Philae
8.Carlos Vela
---------
TT DENMARK 22:45
1.#CometLanding
2.#dkpol
3.Rosetta
4.#Supernatural
http://t.co/WB9v3S4I6t
6.Copenhagen
7.#1tyb3
8.#MurrayFTW
9.Danish
---------
TT PHILIPPINES 04:45
1.#WishkosaPasko
2.#BakitBaGanyan
3.#DiKoKayangTanggapin
4.#CometLanding
5.#HappyBirthdaySandaraPark
6.Blank Space
---------
TT DEUTSCHLAND 22:45
1.#DagiVSTimo
2.#asktaddl
3.#CometLanding
4.#Philae
5.#Rosetta
6.Weihnachten
7.YouTuber
8.WhatsApp


...

@nancywalshcbc are you following the hashtag #CometLanding for all the comet related babble?
---------
Some intermittent @Philae2014 telemetry via ESA/ESOC/DLR @AstroGingerSnap and I LOVE the xkcd pics! #CometLanding http://t.co/iMiakRBvsK
---------
Top story: Rosetta #CometLanding webcast on Livestream http://t.co/vbiVAmjuc7, see more http://t.co/9Jy8bxyteS
---------
Tam Åžuan Kuyruklu YÄ±ldÄ±z'a uzay aracÄ± indirmekle uÄŸraÅŸanlar var #CometLanding BoÅŸver sen Brezilya maÃ§Ä±nÄ±n sonucunu dÃ¼ÅŸÃ¼n...
---------
Top story: Rosetta #CometLanding webcast on Livestream http://t.co/oGPJTcQq89, see more http://t.co/7uHyzPW87P
---------
Wird eigentlich "Final Countdown" abgespielt bei #CometLanding ? :-D
---------
@newscientist Here's our lucky #spaceveg ginger that looks a bit like comet 67P #cometlanding http://t.co/afGTfB5fT9 http://t.co/MJPZKvggVE
---------
This short film is stunning. I'd watch a full-length version: https://t.co/Rd4Oo108Jm #CometLanding #rosettamission #ESA
---------
F

---------
Yes!  Â¡Confirmada la separaciÃ³n! de @Philae2014  #cometlanding
https://t.co/IQ3qdpCAck via @esa
---------
@Silverstrange Thanks for sharing, are you following the #cometlanding?
---------
Waiting in the office for the #CometLanding of @Philae2014!
---------
#CometLanding
---------
In 7 hrs #phelai will land on a comet thats taken 10 yrs to get to. Cannot wrap my head around it. What an achievment! #CometLanding
---------
â€œ@BBCWorld: Follow ongoing coverage of @Philae2014's #Cometlanding attempt http://t.co/0xr9CFLHA8 &amp; our list http://t.co/z5RewFBeXmâ€ super!
---------
â€žSieben Stunden Grauenâ€œ: Der Knochenjob der #Rosetta-Kometenlandung. #CometLanding
http://t.co/7Z6bmUzOVw
---------
Nice BLOG from @ESA on how @ESA_Rosetta is communicating with Earth during #CometLanding
http://t.co/STnw6olADS
---------
Track todayâ€™s historic #CometLanding over @esa 4pm GMT proposed touch down time #goodluck
---------
Congratulations to @esa on the successful deployment of the c

And now we wait for Philea to arrive at the P67 comet in about... 7 hours. At least it'll send some rad pics in about 2 hours. #CometLanding
---------
@esa @ESA_Rosetta  watching all live just to see where the good news comes from ! #Cometlanding http://t.co/apbKCbIqK3
---------
How exciting! We can watch the #cometlanding live on NASA JPL Live http://t.co/7vMInbrB66 @hampstead17 @w1girl
---------
Loving the #CometLanding news, so lucky to have tuned in at exactly the right time. Setting my alarm for later, so exciting!
---------
Crazy suspense today as a tiny robot tries to land on a duckface comet sounding like Predator. #CometLanding https://t.co/8Re5eMuaaF
---------
Meanwhile, in outer space #CometLanding http://t.co/xjqTCiWieF
---------
Philae y #CometLanding, hashtag en EspaÃ±a, porque no todos los dÃ­as se aterriza en un cometa! http://t.co/6l0CHv8uS8 vÃ­a @RosaTristan
---------
Separazione confermata #CometLanding #Rosetta @ESA_Rosetta
---------
Felix Baumgartner - want to try 

---------
Et vous, que diriez-vous ? #CometLanding #PoseToiPhilae #AskCNES http://t.co/sy9MVkyBPP
---------
Here we goooo #rosetta #cometlanding http://t.co/0zv3CgX9Zb http://t.co/Gmwfcw6KGq
---------
Huge congratulations to @ESA_Rosetta! @Philae2014 has successfully separated and is on the way to comet #67P. Bon Voyage! #CometLanding
---------
ESA's Philae lander scheduled to land 7 hours from now. #ESA #Rosetta #Philae #CometLanding http://t.co/91GQzDAbiU
---------
And we've got Lander separation! @ESA_Rosetta 
Now, Gravity, don't be a bitch! #CometLanding
---------
My new hero! @mggtTaylor All the world are talking about your shirt! :) #Rosetta #CometLanding @esa @ESA_Italia
---------
@Philae2014 has officially separated from @ESA_Rosetta! 7 hours until landing! Read the full news here: http://t.co/LzjT2XvTGL #CometLanding
---------
So proud of you all! 10 years of blood sweat and tears and here we go. You are an inspiration for mankind! Thank You #Science #CometLanding
---------
Ka

#CometLanding set your clocks 7 hours from now they will be touching down set for 6 hrs
---------
Philae to Rosetta "it was nice to separate" #CometLanding http://t.co/qQLSGsWuJz
---------
#CometLanding: Why is it important to study comets? Hereâ€™s a concise explainer - http://t.co/QHXC4UbljQ
---------
Watch Rosetta #CometLanding webcast on @livestream: http://t.co/gtgLT7zS9B
---------
Watch Rosetta #CometLanding webcast on @livestream: http://t.co/tdbCyXOmr4
---------
Lander Away: @Philae2014 - 'it's on its path down to the comet' -Flight Director Andrea Accomazzo #CometLanding http://t.co/bYLjgWBLvi
---------
#BNEGlobalCafe
#CometLanding
Hi ðŸ™‹
Plz Listen to The SongðŸ˜ðŸ˜
#Ø§Ø²Ø¹Ù„Ùƒ_ÙˆØ§Ø±Ø¶ÙŠÙƒ 
http://t.co/f6gYa30ryH                  
Abdullah Abdulaziz
***
---------
Two hours 'til communication with lander to verify gear has deployed. #CometLanding  They will beak for lunch now.  :)
---------
All this #CometLanding business is really rather exciting. Bonkers stuff. Hope the 

In [33]:
import json
import pandas as pd 
# Open JSON file.
with open('../code/CometLandingFixed.json', 'r', encoding="utf8") as json_file:
	json_load = json.load(json_file)

# List of hashtags found
hashtagsList = []

# Gets list of hashtags from JSON file.
for j in json_load:
	for i in j['hashtags']:
		hashtagsList.append(i['text'])

json_file.close()
# Gets unique hashtags.
hashtagsSeries = pd.Series(hashtagsList)
hashtagsSeries = hashtagsSeries.unique()

# Creates dataframe.
data = {'Hashtags': hashtagsSeries, 'Frequency': [0] * len(hashtagsSeries)}  
df = pd.DataFrame(data)  

# Finds frequency of each hashtag found.
for i in hashtagsList:
	for index, j in enumerate(df['Hashtags']):
		if(i == j):
			df['Frequency'][index] += 1

# Drops any hashtags with 1 frequency.
#for index, j in enumerate(df['Frequency']):
#	if(j == 1):
#		df.drop(index)
	
# Sort values to descending.
df = df.sort_values(['Frequency'], ascending=False)
			
# Print dataframe.
print(df)


C:\Users\thoma\AppData\Local\Temp/ipykernel_11860/3186434140.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Frequency'][index] += 1


                  Hashtags  Frequency
1             CometLanding      62790
8             cometlanding      13214
20                     67P       7925
10                 Rosetta       5914
0                   Philae       3097
...                    ...        ...
1492  BadSexinFictionAward          1
1491  drinkingaboutmuseums          1
1490                random          1
1489       commentslanding          1
3121                  jp86          1

[3122 rows x 2 columns]
